In [45]:
import pandas as pd
import sqlite3

## Setup the connection to the virtual data base

In [76]:
con = sqlite3.connect("mydb.sqlite3")

In [77]:
df = pd.read_csv('./data/WineQT.csv' )
for col in df.columns:
    df.rename(columns={col:col.replace(" ","_")},inplace=True)
df.to_sql("wines", con, index=False, if_exists="replace")
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


### SELECT 

In [78]:
query = r"""
    SELECT
        volatile_acidity,
        chlorides,
        quality AS class
    from wines
    where
        quality = 5
"""
pd.read_sql_query(query, con)

,volatile_acidity,chlorides,class
0,0.700,0.076,5
1,0.880,0.098,5
2,0.760,0.092,5
3,0.700,0.076,5
4,0.660,0.075,5
...,...,...,...
478,0.690,0.069,5
479,0.715,0.053,5
480,0.460,0.074,5
481,0.600,0.090,5


In [79]:
query = r"""
    SELECT
        volatile_acidity,
        chlorides,
        quality AS class,
        CASE 
            WHEN volatile_acidity < 0.1 THEN "<0.1"
            WHEN volatile_acidity < 0.5 THEN "<0.5"
            ELSE ">=0.5"
        END AS volatile_segment
    from wines
    where
        chlorides > 0.1
"""
pd.read_sql_query(query, con)

,volatile_acidity,chlorides,class,volatile_segment
0,0.61,0.114,5,>=0.5
1,0.32,0.341,6,<0.5
2,0.43,0.106,5,<0.5
3,0.32,0.103,5,<0.5
4,0.49,0.332,6,<0.5
...,...,...,...,...
152,0.70,0.106,5,>=0.5
153,0.68,0.103,6,>=0.5
154,0.63,0.235,5,>=0.5
155,0.36,0.230,6,<0.5


In [80]:
query = r"""
    SELECT
        quality,
        CASE 
            WHEN volatile_acidity < 0.1 THEN "<0.1"
            WHEN volatile_acidity < 0.5 THEN "<0.5"
            ELSE ">=0.5"
        END AS volatile_segment,
        COUNT(quality) as cases
    from wines
    GROUP BY
        1,2
    ORDER BY 1,2
"""
pd.read_sql_query(query, con)

,quality,volatile_segment,cases
0,3,>=0.5,6
1,4,<0.5,6
2,4,>=0.5,27
3,5,<0.5,148
4,5,>=0.5,335
5,6,<0.5,225
6,6,>=0.5,237
7,7,<0.5,112
8,7,>=0.5,31
9,8,<0.5,13


In [81]:
A = df[df['quality'] == 5]
A.to_sql("wines5", con, index=False, if_exists="replace")

B = df[df['quality'] == 6]
B.to_sql("wines6", con, index=False, if_exists="replace")

In [86]:
query = r"""
    SELECT
        *
    from wines5
    UNION
    SELECT
        *
    from wines6
"""
pd.read_sql_query(query, con)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Id
0,5.0,0.380,0.01,1.6,0.048,26.0,60.0,0.99084,3.70,0.75,14.0,6,1270
1,5.0,0.400,0.50,4.3,0.046,29.0,80.0,0.99020,3.49,0.66,13.6,6,1114
2,5.0,0.740,0.00,1.2,0.041,16.0,46.0,0.99258,4.01,0.59,12.5,6,1321
3,5.0,1.040,0.24,1.6,0.050,32.0,96.0,0.99340,3.74,0.62,11.5,5,553
4,5.1,0.470,0.02,1.3,0.034,18.0,44.0,0.99210,3.90,0.62,12.8,6,695
...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,13.8,0.490,0.67,3.0,0.093,6.0,15.0,0.99860,3.02,0.93,12.0,6,347
941,14.3,0.310,0.74,1.8,0.075,6.0,15.0,1.00080,2.86,0.79,8.4,6,544
942,15.5,0.645,0.49,4.2,0.095,10.0,23.0,1.00315,2.92,0.74,11.1,5,554
943,15.6,0.645,0.49,4.2,0.095,10.0,23.0,1.00315,2.92,0.74,11.1,5,557


In [93]:
C = df[['free_sulfur_dioxide','fixed_acidity','density','Id']]
C.to_sql("wines_1", con, index=False, if_exists="replace")

D = df[['alcohol','quality','Id']][df['quality'] == 6]
D.to_sql("wines_2", con, index=False, if_exists="replace")

In [95]:
query = r"""
    SELECT
        A.*,
        B.*
    FROM wines_1 A LEFT JOIN wines_2 B ON A.Id = B.ID
"""
pd.read_sql_query(query, con)

,free_sulfur_dioxide,fixed_acidity,density,Id,alcohol,quality,Id
0,11.0,7.4,0.99780,0,NaN,NaN,NaN
1,25.0,7.8,0.99680,1,NaN,NaN,NaN
2,15.0,7.8,0.99700,2,NaN,NaN,NaN
3,17.0,11.2,0.99800,3,9.8,6.0,3.0
4,11.0,7.4,0.99780,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1138,29.0,6.3,0.99574,1592,11.0,6.0,1592.0
1139,28.0,6.8,0.99651,1593,9.5,6.0,1593.0
1140,32.0,6.2,0.99490,1594,NaN,NaN,NaN
1141,39.0,5.9,0.99512,1595,11.2,6.0,1595.0


In [96]:
query = r"""
    SELECT
        A.*,
        B.*
    FROM wines_1 A INNER JOIN wines_2 B ON A.Id = B.ID
"""
pd.read_sql_query(query, con)

,free_sulfur_dioxide,fixed_acidity,density,Id,alcohol,quality,Id
0,17.0,11.2,0.99800,3,9.8,6,3
1,17.0,7.9,0.99690,19,9.2,6,19
2,21.0,6.9,0.99680,24,9.7,6,24
3,8.0,7.8,0.99640,29,9.8,6,29
4,5.0,7.8,0.99860,35,9.6,6,35
...,...,...,...,...,...,...,...
457,26.0,6.3,0.99314,1590,11.6,6,1590
458,16.0,5.4,0.99402,1591,11.6,6,1591
459,29.0,6.3,0.99574,1592,11.0,6,1592
460,28.0,6.8,0.99651,1593,9.5,6,1593


In [104]:
query = r"""
    SELECT
        MIN(A.free_sulfur_dioxide) AS min_free_sulfur_dioxide,
        MAX(B.alcohol) AS max_alcohol,
        COALESCE(A.quality,B.quality)
    FROM wines_1 A LEFT JOIN wines_2 B ON A.Id = B.ID
    GROUP BY quality
"""
pd.read_sql_query(query, con)

DatabaseError: Execution failed on sql '
    SELECT
        MIN(A.free_sulfur_dioxide) AS min_free_sulfur_dioxide,
        MAX(B.alcohol) AS max_alcohol,
        COALESCE(A.quality,B.quality)
    FROM wines_1 A LEFT JOIN wines_2 B ON A.Id = B.ID
    GROUP BY quality
': no such column: A.quality